In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("trip_count_by_zone_sql").getOrCreate()

In [5]:
directory = "/Users/gimhyeonjeong/data-engineering/01-spark/data"
trip_file = "fhvhv_tripdata_2020-03.csv"
zone_file = "taxi+_zone_lookup.csv"

In [7]:
trip_data = spark.read.csv(f"file:///{directory}/{trip_file}", inferSchema = True, header= True)
zone_data = spark.read.csv(f"file:///{directory}/{zone_file}", inferSchema = True, header= True)

In [8]:
trip_data.show(5)

+---+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|_c0|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+---+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+--

In [9]:
zone_data.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [10]:
trip_data.createOrReplaceTempView("trip_data")
zone_data.createOrReplaceTempView("zone_data")

In [11]:
spark.sql("select *from trip_data limit 5").show()

+---+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|_c0|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+---+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+--

In [12]:
spark.sql("select *from zone_data limit 5").show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+



In [14]:
spark.sql("select zone_data.Borough from trip_data join zone_data on trip_data.PULocationID = zone_data.LocationID").show()

+---------+
|  Borough|
+---------+
|    Bronx|
|    Bronx|
|Manhattan|
|Manhattan|
| Brooklyn|
|Manhattan|
| Brooklyn|
| Brooklyn|
| Brooklyn|
|    Bronx|
|    Bronx|
|    Bronx|
|Manhattan|
|Manhattan|
|    Bronx|
|    Bronx|
|Manhattan|
|Manhattan|
|Manhattan|
|   Queens|
+---------+
only showing top 20 rows



In [18]:
spark.sql("select borough, count(*) as trips from \
(select zone_data.Borough as borough \
from trip_data join zone_data on trip_data.PULocationID = zone_data.LocationID)\
group by borough").show()

+-------------+-------+
|      borough|  trips|
+-------------+-------+
|       Queens|2437394|
|          EWR|    362|
|      Unknown|    845|
|     Brooklyn|3735765|
|Staten Island| 178818|
|    Manhattan|4953147|
|        Bronx|2086597|
+-------------+-------+



In [19]:
spark.sql("select borough, count(*) as trips from \
(select zone_data.Borough as borough \
from trip_data join zone_data on trip_data.DOLocationID = zone_data.LocationID)\
group by borough").show()

+-------------+-------+
|      borough|  trips|
+-------------+-------+
|       Queens|2468416|
|          EWR|  65066|
|      Unknown| 387760|
|     Brooklyn|3696684|
|Staten Island| 177727|
|    Manhattan|4553783|
|        Bronx|2043492|
+-------------+-------+

